# Julia is roughly as fast as C

due to
  1. just-in-time compilation
  2. being smart about types


## Define a simple function and run it twice

In [1]:
f(x) = x^3 - 2

f (generic function with 1 method)

In [2]:
@time f(0.3);
@time f(0.4);

  0.004567 seconds (1.20 k allocations: 74.862 KiB)
  0.000002 seconds (5 allocations: 176 bytes)


In [3]:
@time f(0.3)

  0.000002 seconds (5 allocations: 176 bytes)


-1.973

** The second evaluation is 4000 times faster than the first! Why? Just-in-time compilation.**

In [4]:
@code_lowered(f(7.0))   # show f(x) in Julia's abstract syntax tree

CodeInfo(:(begin 
        nothing
        return (Base.literal_pow)(Main.^, x, (Core.apply_type)(Base.Val, 3)) - 2
    end))

In [5]:
@code_llvm f(7.0)       # show f(x) in LLVM (compiler) intermediate language


define double @julia_f_60971(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, %0
  %3 = fadd double %2, -2.000000e+00
  ret double %3
}


In [6]:
@code_native f(7.0)     # show f(x) in Intel IA-64 assembly language

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movapd	%xmm0, %xmm1
	mulsd	%xmm1, %xmm1
	mulsd	%xmm0, %xmm1
	movabsq	$139680790852296, %rax  # imm = 0x7F09F7EBB2C8
	addsd	(%rax), %xmm1
	movapd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


## Compare intermediate code for f(x) on floats and ints

In [ ]:
@code_llvm(f(7.0))

In [ ]:
@code_llvm(f(7))

#### type inference: figuring out the types of untyped variables, crucial for just-in-time compilation

In [ ]:
sqrt(-1)

## Redefine f(x), iterate it, and time for comparison to C

In [ ]:
# define function that, given an f, returns iterated function f^N
function iterator(f, N)
    
    # construct f^N
    function fN(x)
      for i ∈ 1:N             
        x = f(x)
      end
      x
    end    
    
    fN     # return f^N
end

# define logistic map function
f(x) = 4*x*(1-x)


# use iterator function to constuct millionth iterate of logistic map
fᴺ  = iterator(f, 1000000)  

In [ ]:
@time x = fᴺ(0.34);
@time x = fᴺ(0.67);

In [ ]:
@show x;

## Timing of equivalent C++ code

note: starting semicolon tells Julia to execute Unix shell code

In [ ]:
; pwd

In [ ]:
; cat fmillion.cpp

In [ ]:
; g++ -O3 -o fmillion fmillion.cpp

### Execution time for C++

In [ ]:
; fmillion 0.67

### Execution time for Julia

In [ ]:
print("t=")
@time x = fᴺ(0.67);
@show x;

Speeds are comparable. Sometimes Julia is faster, sometimes C.

## Julia benchmarks versus other languages


![Julia benchmarks](http://channelflow.org/juliabenchmarks.png  " Julia benchmarks")

http://julialang.org/benchmarks
